##### <h1 align=center><font size=5>SAM Python Capstone Final Project</font></h1>

## Description

<p>The final project is an automatic grading system.  The goal is open each of the files included in
the compressed file</p>

### Instructions
<div class="alert alert-block alert-info" style="margin-top: 20px">

1.  [Create a class roster by parsing the file names](#0)<br>
2.  [Create a student exam GPA](#2)<br>
3.  [Create a summary report](#4) <br>
4.  [Create an input screen for weighted average](#6) <br> 
5.  [Apply weighted average- create report](#8) <br>  
</div>
<hr>

### Class roster<a id="0"></a>
#### 10 points
#### Create a file called 'PYTHON_PTI_CAPSTONE_CLASS_ROSTER.TXT'
#### In should contain an unduplicate list of class names
#### 1. Lastname, Firstname
#### 2. Lastname,Firstname
#### NOTE: The names should be in alphabetical order by last name, first name and in proper case
#### You must parse the file names to get the student names:  CAPSTONE_lastname_firstname_QUIZ.......txt

In [2]:
# ------- CREATING CLASS ROSTER ------- 10 Pts ------------------------------------------------------------
roster = 'python_pti_capstone_class_roster.txt'

import os
file_names = os.listdir('./PYTHON_PTI_CAPSTONE_PROJECT_FILES')

# List of student names sorted by last name
studen_roster = sorted(set([
    # parsed 
    ' '.join(student.lower().replace('capstone_','').split("_")[:2]) for student in file_names
    ]))

# Write them to the specified filename and close the file
file = open(roster, 'w')
[file.write(student.split()[0].capitalize()+ ' ' + student.split()[1].capitalize() +'\n') for student in studen_roster if student != 'final exam' and student != 'quiz key']
file.close()
#----------------------------------------------------------------------------------------------------------

In [3]:
# Create function that takes in the name of a file and returns the grade
from pathlib import Path
def grade_assignment(assignment_name='CAPSTONE_davis_FRANK_QUIZ_1_LESSON_1.TXT'):
    file_names = os.listdir('./PYTHON_PTI_CAPSTONE_PROJECT_FILES')

    # List of student names sorted by last name
    keys = sorted(set([
        key for key in file_names if 'key' in key.lower()
    ]))

    # Use the last character+file_extension to match the key to the assignment provided
    assignment_key = [key for key in keys if key.split('_')[-1].lower() == assignment_name.split('_')[-1].lower()]
    if len(assignment_key) == 0:
        assignment_key = 'CAPSTONE_FINAL_EXAM_KEY.TXT'
    else:
        assignment_key = assignment_key[0]

    # read the students answers
    student_answers = open(Path('./PYTHON_PTI_CAPSTONE_PROJECT_FILES/' + assignment_name), 'r')
    student_answers = [answer.strip() for answer in student_answers]
    # read the key answers
    key_answers = open(Path('./PYTHON_PTI_CAPSTONE_PROJECT_FILES/' + assignment_key), 'r')
    key_answers = [answer.strip() for answer in key_answers]

    # remember how many questions/points on the assignment
    number_of_questions = len(key_answers)
    grade = 0

    # give the student a point for every question they got correct
    for question in range(number_of_questions):
        # Avoiding index errors by caused by mismatching lengths
        try:
            # print(key_answers[question],student_answers[question])
            if student_answers[question] == key_answers[question]:
                grade += 1
        except:
            # student had less answers than the key did...embarrassing
            pass

    # return their grade EXAMPLE 90.0
    return (grade/number_of_questions)*100

grade_assignment()

80.0

In [4]:
# BUILDING MASTER GRADE DICTIONARY students:{student:{assignments:{1:grade,2:grade,3:grade}}}
import pickle
student_dictionary = {}

# student names in lower case
students_lower = []
file = open(roster,'r')
for line in file:
    students_lower.append(line.strip())

# get list of all assignements (sort assignments by name for cleanliness)
file_names = os.listdir('./PYTHON_PTI_CAPSTONE_PROJECT_FILES')
assignments = sorted([assignment for assignment in file_names])

# Get all assignments per student and create the dictionary
for student in students_lower:   
    student_dictionary[student] = {
        # get assignment grade during dict construction
        item:grade_assignment(item) for item in assignments if '_'.join(student.lower().split()) in item.lower()
        }

    # update the students dictionary item with... the average of their scores
    student_dictionary[student]['mean'] = \
    int(sum([grade for _ ,grade in student_dictionary[student].items()])/6)

    student_dictionary[student]['weighted_grade'] = 0
    
# save the student_dictionary via Pickle
with open('student_data.dat', 'wb') as file:
    pickle.dump(student_dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)

### Class roster with mean test scores<a id="2"></a>
#### 30 points
#### Create a file called 'PYTHON_PTI_CAPSTONE_CLASS_ROSTER_GPA.TXT'
#### In should contain an unduplicate list of student names their graded scores and their mean
#### 1. Lastname, Firstname - Test Score 1, Test Score 2, ... Final Exam Score, Mean
#### 2. Lastname, Firstname - Test Score 1, Test Score 2, ... Final Exam Score, Mean
#### NOTE: The names should be in alphabetical order by last name, first name and in proper case
#### Use the file 'CAPSTONE_QUIZ_KEY_LESSON_1' through 5 to score the exams and calculate the mean

In [5]:
filename = 'PYTHON_PTI_CAPSTONE_CLASS_ROSTER_GPA.TXT'

# -----CLass Roster With Mean Scores----------30 pts------------------------------
file = open(filename, 'w')
for student in student_dictionary.items():
    # put the students grades into a line
    line = [str(grade) for _,grade in student[1].items()][:-1]
    # arrange the final grade to appear before the mean, and insert the students name
    line.insert(-1,line.pop(0)), line.insert(0,student[0])

    # Write the line to the specified file
    file.write(' '.join(line)+'\n')
file.close()

### Class summary report<a id="4"></a>
#### 15 points
#### Create a file called 'PYTHON_PTI_CAPSTONE_SUMMARY REPORT.TXT'
#### It should contain the following
#### 1. There were X students
#### 2. Exam 1 - mean, standard deviation
#### 3. Exam 2 - mean, standard deviation
#### 4. Exam 3 - mean, standard deviation
#### 5. Exam 4 - mean, standard deviation
#### 6. Exam 5 - mean, standard deviation
#### 7. Final exam - mean, standard deviation

In [6]:
filename = 'PYTHON_PTI_CAPSTONE_SUMMARY REPORT.TXT'
students_scores = 'PYTHON_PTI_CAPSTONE_CLASS_ROSTER_GPA.TXT'

import statistics
import pickle

# Read in the students grades
file = open(students_scores, 'r')
grades = [line.strip().split(' ')[2:-1] for line in file]
# get a list of exam key names
file_names = os.listdir('./PYTHON_PTI_CAPSTONE_PROJECT_FILES')

# List of the grading key file names
keys = sorted(set([
    key for key in file_names if 'key' in key.lower()
]))

# arrage the final exam to the end of the list
keys.append(keys.pop(0))

# Find the mean
def mean_test_scores(score_list):
    return int(sum(score_list)/len(score_list))

# calc standard deviation
def standard_deviation(grade_list):
    return int(statistics.stdev(grade_list))
file.close()


# dictionary of assignments and a list of students scores on them with mean and standard deviation
assignment_scores = {
    name : {
        # weight placeholder for the assignments
        'weight': 0, 

        # all the scores for that assignment
        'scores': [grade[index] for grade in grades],

        # pass the list of scores to mean calculation function
        'mean': mean_test_scores([float(grade[index]) for grade in grades]),

        # pass the list of scores to standard deviation function
        'standard_deviation': standard_deviation([float(grade[index]) for grade in grades])
        } for index, name in enumerate(keys)
    }
    
# Pickle assignment_scores for later use
with open('exam_data.dat', 'wb') as file:
    pickle.dump(assignment_scores, file, protocol=pickle.HIGHEST_PROTOCOL)

# ------ Class Summary Report ----------15 Pts--------------------------
file = open(filename, 'w')
for field in assignment_scores:
    # write each assignment to the file with its mean score, and its standard deviation
    file.write(f"{field} {assignment_scores[field]['mean']} {assignment_scores[field]['standard_deviation']}\n")
file.close()
# -----------------------------------------------------------------------

### Class summary report<a id="6"></a>
#### 15 points
#### Create an input screen which saves the following weights: Exam 1 through 5 are 15 points and final is 25 points
#### Exam #
#### Weight
#### NOTE: BONUS - Pickle the data as a dictionary instead of creating a file 'PYTHON_PTI_CAPSTONE_WEIGHTS.TXT'

In [7]:
# -----Input Screen for Exam Weights----15 Pts--------------(Bonus Pickle)

exam_weight = 5 #input('Enter weight adjustment for Exams 1-5')
final_weight = 15 #input('Enter weight adjustment for Final')

# Load the previously pickled exam data
file = open('exam_data.dat', 'rb')
exam_data = pickle.load(file)
file.close()
# Update dictionary weights with proper values
for exam in exam_data:
    if 'quiz' in exam.lower():
        exam_data[exam]['weight'] = 5
    if 'final' in exam.lower():
        exam_data[exam]['weight'] = 15

# Save the pickle
with open('exam_data.dat', 'wb') as file:
    pickle.dump(exam_data, file, protocol=pickle.HIGHEST_PROTOCOL)
# ------------------------------------------------------------------------

### Final grade for roster<a id="8"></a>
#### 30 points
#### List every student from highest to lowest final grade without weights PYTHON_PTI_CAPSTONE_GRADES.TXT
#### List every student from highest to lowest final grade with weights  PYTHON_PTI_CAPSTONE_WEIGHTED_GRADES.TXT
#### Use must open and use the PYTHON_PTI_CAPSTONE_WEIGHTS.TXT File created above
#### 1. Last name, first name - Final grade
#### 2. Last name, first name - Final grade
#### Add a summary
#### Mean, Mode, Median, Standard deviation
#### How many students are within 1 standard deviation of the mean
#### How many students are within 2 standard deviations of the mean
#### How many students are within 3 standard deviations of the mean
#### Is this a normal distribution?

In [8]:
import pickle
import statistics
# load our big exam and student data dictionaries
with open('student_data.dat', 'rb') as file:
    student_data = (pickle.load(file))
with open('exam_data.dat', 'rb') as file:
    exam_data = (pickle.load(file))
# Making a better dictionary to hold more assignment data (Should've done this in the beginning....)
for student in student_data:
    # initializing new dictionary keys
    student_data[student]['weighted_grade'] = []
    student_data[student]['unweighted_grade'] = []
    student_data[student]['mean_student_gpa_weighted'] = 0
    student_data[student]['mean_student_gpa_unweighted'] = 0
    scores = []
    weighted_scores = []
    # O(n)^2 :)
    for index,exam in enumerate(exam_data):
        student_grade = list(student_data[student].items())[index][1]
        
        current_exam = list(student_data[student].items())[index][0]
        exam_weight = exam_data[exam]['weight']
        #print(student_grade, exam_weight)
        student_data[student][current_exam] = {
            'points_possible':exam_data[exam]['weight'],
            'points_earned':exam_data[exam]['weight'] * student_grade/100,
            'score': student_grade,
            'weighted_score': student_grade + exam_weight,
            'assignment_weight': exam_weight,
            'class_average_score': exam_data[exam]['mean'],
            'class_average_score_weighted': [],
            'standard_deviation': exam_data[exam]['standard_deviation']
        }
        # Add to grade scores
        student_data[student]['weighted_grade'].append(student_data[student][current_exam]['weighted_score'])
        student_data[student]['unweighted_grade'].append(student_data[student][current_exam]['score'])
    
    # calculate mode of students grades
    student_data[student]['student_grade_mode_weighted'] = int(statistics.mode(student_data[student]['weighted_grade']))
    student_data[student]['student_grade_mode_unweighted'] = int(statistics.mode(student_data[student]['unweighted_grade']))
    # Weighted and unweighted average gpa per student
    student_data[student]['mean_student_gpa_weighted'] = int(statistics.mean(student_data[student]['weighted_grade']))
    student_data[student]['mean_student_gpa_unweighted'] = int(statistics.mean(student_data[student]['unweighted_grade']))
    # Weighted and unweighted mean grade per student
    student_data[student]['student_median_grade_weighted'] = int(statistics.median(student_data[student]['weighted_grade']))
    student_data[student]['student_median_grade_unweighted'] = int(statistics.median(student_data[student]['unweighted_grade']))
    # Standard deviation unweighted and weighted
    student_data[student]['standard_deviation_weighted'] = int(statistics.stdev(student_data[student]['weighted_grade']))
    student_data[student]['standard_deviation_unweighted'] = int(statistics.stdev(student_data[student]['unweighted_grade']))
    # calc grade averages
    student_data[student]['weighted_grade'] = int(statistics.mean(student_data[student]['weighted_grade']))
    student_data[student]['unweighted_grade'] = int(statistics.mean(student_data[student]['unweighted_grade']))

# save the dictionary update
with open('student_data.dat', 'wb') as file:
    pickle.dump(student_data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
# List students by final grade (hightest to lowest) without weights  √
# List students by final grade (hightest to lowest) with weights  √
# Last name, First name - Final grade  √
#  include summary:
#   mean, mode, median, standard deviation

# how many students are with 1,2 and 3 of the standard deviation of the mean


# Unpickle the student master dictionary
with open('student_data.dat', 'rb') as file:
    student_data = (pickle.load(file))
file.close()


filename = 'PYTHON_PTI_CAPSTONE_GRADES.TXT'
file = open(filename, 'w')
# Sorting and displaying students by unweighted grade descending
sorted_unweighted = {student : int((list(student_data[student].items())[0][1]['score'])) for student in student_data}
sorted_unweighted = dict(sorted(sorted_unweighted.items(), key=lambda item: item[1],reverse=True)) 
# Write the students sorted by grade to file specified
file.write('--SORTED UNWEIGHTED--\n')
for index, student in enumerate(sorted_unweighted):
   file.write(f"{index+1}. {student}: {sorted_unweighted[student]}\n")
file.close()


filename = 'PYTHON_PTI_CAPSTONE_WEIGHTED_GRADES.TXT'
file = open(filename, 'w')
# Sorting and displaying students by unweighted grade ascending
sorted_weighted = {student : int((list(student_data[student].items())[0][1]['weighted_score'])) for student in student_data}
sorted_weighted = dict(sorted(sorted_weighted.items(), key=lambda item: item[1],reverse=True))
# Write the students sorted by grade to file specified
file.write('--SORTED WEIGHTED--\n')
for index, student in enumerate(sorted_weighted):
   file.write(f"{index+1}. {student}: {sorted_weighted[student]}\n")
file.close()


# Summary (Mean, Median, Mode, Standard Deviation)
filename = 'PYTHON_PTI_CAPSTONE_SUMMARY.TXT'
file = open(filename, 'w')
for student in student_data:
   # Formatting student data into a log and saving it to file
   file.write(f"""{student}: \n\tGrade: {student_data[student]['unweighted_grade']} \n\tWeighted Grade: {student_data[student]['weighted_grade']}
   \tMode Unweighted: {student_data[student]['student_grade_mode_unweighted']} \n\tMode Weighted: {student_data[student]['student_grade_mode_weighted']}
   \tMedian Unweighted: {student_data[student]['student_median_grade_unweighted']} \n\tMedian Weighted: {student_data[student]['student_median_grade_weighted']}
   \tStandard Deviation Unweighted: {student_data[student]['standard_deviation_unweighted']} \n\tStandard Deviation Weighted: {student_data[student]['standard_deviation_weighted']}\n\n""")
file.close()